In [0]:
%sql
CREATE OR REPLACE TABLE genealogy.silver_person AS
SELECT
  record_xref              AS person_gedcom_id,
  MAX(CASE WHEN tag = 'SEX'  THEN value END) AS sex,
  MAX(CASE WHEN tag = 'NAME' THEN value END) AS name_raw,
  MAX(CASE WHEN tag = 'GIVN' THEN value END) AS given_name,
  MAX(CASE WHEN tag = 'SURN' THEN value END) AS surname,
  MAX(CASE WHEN tag = 'NSFX' THEN value END) AS name_suffix,
  MAX(CASE WHEN tag = 'DSCR' THEN value END) AS person_description,
  MAX(CASE WHEN tag = 'NOTE' AND level = 1 THEN value END) AS person_note
FROM genealogy.bronze_gedcom_with_record
WHERE record_xref LIKE '@I%@'
GROUP BY record_xref;


In [0]:
%sql
SELECT COUNT(*) FROM genealogy.silver_person;


In [0]:
%sql
CREATE OR REPLACE TABLE genealogy.silver_family AS
SELECT
  record_xref AS family_gedcom_id,

  MAX(CASE WHEN tag = 'HUSB' THEN value END) AS husband_gedcom_id,
  MAX(CASE WHEN tag = 'WIFE' THEN value END) AS wife_gedcom_id,
  MAX(CASE WHEN tag = '_SREL' THEN value END) AS relationship_type

FROM genealogy.bronze_gedcom_with_record
WHERE record_xref LIKE '@F%@'
GROUP BY record_xref;


In [0]:
%sql
SELECT * FROM genealogy.silver_family LIMIT 20;


In [0]:
%sql
CREATE OR REPLACE TABLE genealogy.silver_family_child AS
SELECT
  b.record_xref AS family_gedcom_id,
  b.value       AS child_gedcom_id,
  coalesce(b.line_no, c.parent_line_no) AS child_line_no,
  COALESCE(MAX(CASE WHEN c.tag = '_FREL' THEN c.value END),'biological') AS father_relationship_type,
  COALESCE(MAX(CASE WHEN c.tag = '_MREL' THEN c.value END), 'biological') AS mother_relationship_type
FROM genealogy.bronze_gedcom_with_record b
LEFT JOIN genealogy.bronze_gedcom_enriched c
  ON c.parent_line_no = b.line_no
WHERE
  b.record_xref LIKE '@F%@'
  AND b.tag = 'CHIL'
GROUP BY b.record_xref, b.value, child_line_no

In [0]:
%sql
SELECT * FROM genealogy.silver_family_child LIMIT 20;


In [0]:
%sql
CREATE OR REPLACE TABLE genealogy.silver_event_participant AS
SELECT
  e.event_id,
  e.record_xref        AS person_id,
  'principal'          AS role,
  'recorded'           AS data_source
FROM genealogy.silver_event e
WHERE e.record_xref IS NOT NULL
AND e.record_xref LIKE '@I%'
AND e.event_type NOT IN ('NAME', 'SEX', 'FACT');


In [0]:
%sql
INSERT INTO genealogy.silver_event_participant
SELECT
  e.event_id,
  f.husband_gedcom_id,
  'husband',
  'recorded'
FROM genealogy.silver_event e
JOIN genealogy.silver_family f
  ON e.record_xref = f.family_gedcom_id
WHERE e.event_type = 'MARR'
  AND f.husband_gedcom_id IS NOT NULL;

INSERT INTO genealogy.silver_event_participant
SELECT
  e.event_id,
  f.wife_gedcom_id,
  'wife',
  'recorded'
FROM genealogy.silver_event e
JOIN genealogy.silver_family f
  ON e.record_xref = f.family_gedcom_id
WHERE e.event_type = 'MARR'
  AND f.wife_gedcom_id IS NOT NULL;


In [0]:
%sql
INSERT INTO genealogy.silver_event_participant
SELECT
  e.event_id,
  f.husband_gedcom_id,
  'father',
  'derived'
FROM genealogy.silver_event e
JOIN genealogy.silver_family_child fc
  ON e.record_xref = fc.child_gedcom_id
JOIN genealogy.silver_family f
  ON f.family_gedcom_id = fc.family_gedcom_id
WHERE e.event_type in ('BIRT', 'BAP', 'CHR')
  AND f.husband_gedcom_id IS NOT NULL
  AND fc.father_relationship_type = 'biological';

INSERT INTO genealogy.silver_event_participant
SELECT
  e.event_id,
  f.wife_gedcom_id,
  'mother',
  'derived'
FROM genealogy.silver_event e
JOIN genealogy.silver_family_child fc
  ON e.record_xref = fc.child_gedcom_id
JOIN genealogy.silver_family f
  ON f.family_gedcom_id = fc.family_gedcom_id
WHERE e.event_type in ('BIRT', 'BAP', 'CHR')
  AND f.wife_gedcom_id IS NOT NULL
  AND fc.mother_relationship_type = 'biological';

In [0]:
%sql
CREATE OR REPLACE TABLE genealogy.silver_relationship AS
-- spouses
SELECT
  husband_gedcom_id        AS person_id_1,
  wife_gedcom_id           AS person_id_2,
  'spouse'          AS relationship_type,
  family_gedcom_id
FROM genealogy.silver_family
WHERE husband_gedcom_id IS NOT NULL
  AND wife_gedcom_id IS NOT NULL

UNION ALL

-- parent → child (father)
SELECT
  f.husband_gedcom_id,
  fc.child_gedcom_id,
  'parent',
  f.family_gedcom_id
FROM genealogy.silver_family f
JOIN genealogy.silver_family_child fc
  ON f.family_gedcom_id = fc.family_gedcom_id
WHERE f.husband_gedcom_id IS NOT NULL

UNION ALL

-- parent → child (mother)
SELECT
  f.wife_gedcom_id,
  fc.child_gedcom_id,
  'parent',
  f.family_gedcom_id
FROM genealogy.silver_family f
JOIN genealogy.silver_family_child fc
  ON f.family_gedcom_id = fc.family_gedcom_id
WHERE f.wife_gedcom_id IS NOT NULL;


In [0]:
%sql
CREATE OR REPLACE TABLE genealogy.silver_person_name AS
SELECT
  person_gedcom_id,
  'as_recorded'                         AS name_type,
  name_raw,

  -- surname: text inside / /
  NULLIF(
    trim(regexp_extract(name_raw, '/([^/]+)/', 1)),
    ''
  ) AS surname,

  -- given name: everything except /surname/
  NULLIF(
    trim(regexp_replace(name_raw, '/[^/]*/', '')),
    ''
  ) AS given_name

FROM genealogy.silver_person;


In [0]:
%sql
CREATE OR REPLACE TABLE genealogy.silver_repository AS
SELECT
  record_xref              AS repo_gedcom_id,
  MAX(CASE WHEN tag = 'NAME' THEN value END) AS repo_name,
  MAX(CASE WHEN tag = 'ADDR' THEN value END) AS repo_addr,
  MAX(CASE WHEN tag = 'NOTE' THEN value END) AS repo_note
FROM genealogy.bronze_gedcom_with_record
WHERE record_xref LIKE '@R%@'
GROUP BY record_xref;

In [0]:
%sql
CREATE OR REPLACE TABLE genealogy.silver_tag AS
SELECT
  record_xref              AS tag_gedcom_id,
  MAX(CASE WHEN tag = 'NAME' THEN value END) AS tag_name,
  MAX(CASE WHEN tag = 'NOTE' THEN value END) AS tag_note,
  MAX(CASE WHEN tag = '_MTCAT' THEN value END) AS tag_category_gedcom_id
FROM genealogy.bronze_gedcom_with_record
WHERE record_xref LIKE '@T%@'
GROUP BY record_xref;

In [0]:
%sql
CREATE OR REPLACE TABLE genealogy.silver_tag_category AS
SELECT
  record_xref              AS tag_category_gedcom_id,
  MAX(CASE WHEN tag = 'NAME' THEN value END) AS tag_category_name
FROM genealogy.bronze_gedcom_with_record
WHERE record_xref LIKE '@C%@'
GROUP BY record_xref;

In [0]:
%sql
CREATE OR REPLACE TABLE genealogy.silver_object AS
SELECT
  record_xref              AS obj_category_gedcom_id,
  MAX(CASE WHEN tag = 'TYPE' THEN value END) AS obj_type,
  MAX(CASE WHEN tag = '_MTYPE' THEN value END) AS obj_media_type,
  MAX(CASE WHEN tag = 'TITL' THEN value END) AS obj_title,
  MAX(CASE WHEN tag = 'DATE' THEN value END) AS obj_date,
  MAX(CASE WHEN tag = '_DSCR' THEN value END) AS obj_description,
  MAX(CASE WHEN tag = '_OID' THEN value END) AS obj_id,
  MAX(CASE WHEN tag = '_CREA' THEN value END) AS obj_created
FROM genealogy.bronze_gedcom_with_record
WHERE record_xref LIKE '@O%@'
GROUP BY record_xref;

In [0]:
%sql
CREATE OR REPLACE TABLE genealogy.silver_person_tag AS
SELECT
  record_xref AS person_gedcom_id
  , value AS tag_gedcom_id
  , line_no
FROM genealogy.bronze_gedcom_with_record
WHERE tag = '_MTTAG'
AND level = 1;

In [0]:
%sql
CREATE OR REPLACE TABLE genealogy.silver_object_link AS
select
  ge.record_xref
  , value as obj_gedcom_id
  , line_no
  , ge.event_line_no
  , ge.citation_root_line_no
  , case when level = 1 then 'person'
    when level = 2 then 'event'
    when level = 3 then 'source_citation'
    end as link_type
  , e.event_id
  , es.source_xref
from genealogy.bronze_gedcom_enriched ge
left join genealogy.silver_event e on ge.event_line_no = e.event_line_no
left join genealogy.silver_event_source es on ge.citation_root_line_no = es.citation_root_line_no
where tag = 'OBJE'
and level > 0;